In [1]:
import folium
import json

# Load the JSON file with your polygons
with open('2020.json') as f:
    data = json.load(f)

# Initialize the map centered around the first point in your JSON file
first_polygon = data['features'][0]['geometry']['coordinates'][0][0][0]
m = folium.Map(location=[first_polygon[1], first_polygon[0]], zoom_start=4)

# Loop through the features and add each to the map
for feature in data['features']:
    # Make sure the feature type is a MultiPolygon
    if feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            # The polygon must be inverted due to folium's (lat, lon) format
            inverted_polygon = [[(lat, lon) for lon, lat in linear_ring] for linear_ring in polygon]
            folium.Polygon(
                locations=inverted_polygon,
                color='blue',
                fill=True,
                fill_color='cyan',
                fill_opacity=0.5,
            ).add_to(m)

# Show the map
m.save('map.html')
m

In [4]:
# Initialize variables to store the largest number of coordinates and the corresponding polygon
largest_coords_count = 0
largest_polygon_coords = []

# Loop through the features to find the polygon with the most coordinates
for feature in data['features']:
    # In GeoJSON, each feature's geometry can contain multiple polygons (in case of a MultiPolygon)
    for polygon in feature['geometry']['coordinates']:
        # Flatten the list of coordinates in case of a MultiPolygon
        # Count the total number of coordinates across all parts of the polygon
        coords_count = sum(len(part) for part in polygon)
        # Update if the current polygon has more coordinates than the previous maximum
        if coords_count > largest_coords_count:
            largest_coords_count = coords_count
            largest_polygon_coords = polygon  # This is a list of lists now

# Find the center of the largest polygon for map initialization
# For simplicity, just using the first coordinate of the first polygon part
center = largest_polygon_coords[0][0]

# Swap the coordinates because Folium uses (lat, lon) format
center = [center[1], center[0]]

# Initialize the map with the center of the largest polygon
m = folium.Map(location=center, zoom_start=12)

# Add the largest polygon to the map
# Folium requires a list of (lat, lon) tuples, hence the need to swap and flatten the coordinates
for part in largest_polygon_coords:
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in part],  # Swapping each coordinate
        color='blue',
        fill=True,
        fill_color='cyan'
    ).add_to(m)

# Save the map to an HTML file
m.save('largest_polygon_map.html')

# If you want to display the map here in a Jupyter notebook, you can simply output the map object.
m